# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

/Users/jamesrushing/opt/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pyscf/lib/misc.py:50: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
#from IPython.display import HTML
#HTML('''<script>
#var code_show_err = false; 
#var code_toggle_err = function() {
# var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
# var stderr = Array.from(stderrNodes)
# if (code_show_err){
#     stderr.forEach(ele => ele.style.display = 'block');
# } else {
#     stderr.forEach(ele => ele.style.display = 'none');
# }
# code_show_err = !code_show_err
#} 
#document.addEventListener('DOMContentLoaded', code_toggle_err);
#</script>
#To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
gsd_file = "/Users/jimmy/cmelab/data/ITIC/workspace/35e3dc4320d6ed3424bdba79fae77bce/trajectory.gsd"
g = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[1]
    end_snap = f[gsd_length - 1]

    
box = start_snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jimmy/cmelab/data/ITIC/workspace/35e3dc4320d6ed3424bdba79fae77bce/trajectory.gsd'

below is some scratch for breaking chromophores into pieces of smiles strings. may be useful down the road but for now its easier to explicity prescibe atom indeces for your chromo

In [ ]:
#smarts_str = "c1ccccc1"
#aaids=[]
#aaids_pent = chromophores.get_chromo_ids_smiles(snap, "c1sc2ccsc2c1", amber_dict)
#comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)

#aaids_benzene = chromophores.get_chromo_ids_smiles(snap, smarts_str, amber_dict)
#aaids_thiothene = chromophores.get_chromo_ids_smiles(snap, "c1sc(C)cc1", amber_dict)
#aaids.extend(aaids_benzene)
#aaids.extend(aaids_thiothene)
#aaids.extend(aaids_pent)


the indexes below came from viewing the single molecule trajectory in vmd and clicking on the molecules that I like for my chromos. this cell visualizes those chosen

In [ ]:
chromo_ids = np.array([0,1,2,4,6,7,10,11,12,13,19,20,22,23,24,25,27,28,29,30,31,32,91,92,93,94,97,98,99,100,101,102,161,162,163,165,166,168,169,170,171,172,178,179,180,181,177,175,176,174,173,17,15,18,16,14,13])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

Trying to get homo and lumo for a single molecule of itic below. using functions in execute_qcc.py. here we use all molecule ids from start_snap.particles.N to get homo lumos for sinlge molecule gsd

displayed : 

HOMO-1, HOMO, LUMO, LUMO+1 above. 

In [ ]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(start_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


In [ ]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(end_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)

In [ ]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


below i want to get the homolumos for just that back bone as taken from a single molecule gsd

In [ ]:
qcc_input = eqcc.write_qcc_inp(end_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)
